# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [49]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter
import json
from numerize import numerize
from statistics import mean

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [9]:
url = 'https://scanner.tradingview.com/america/scan'
payload = {"filter":[{"left":"type","operation":"in_range","right":["stock","dr","fund"]},{"left":"subtype","operation":"in_range","right":["common","foreign-issuer","","etf","etf,odd","etf,otc","etf,cfd"]},{"left":"exchange","operation":"in_range","right":["AMEX","NASDAQ","NYSE"]},{"left":"is_primary","operation":"equal","right":True},{"left":"active_symbol","operation":"equal","right":True}],"options":{"lang":"en"},"markets":["america"],"symbols":{"query":{"types":[]},"tickers":[]},"columns":["logoid","name","close","change_from_open","change|1","change|5","change|15","change|60","change|240","change","change|1W","change|1M","Perf.W","Perf.1M","Perf.3M","Perf.6M","Perf.YTD","Perf.Y","average_volume_90d_calc","description","type","subtype","update_mode","pricescale","minmov","fractional","minmove2","currency","fundamental_currency_code"],"sort":{"sortBy":"name","sortOrder":"asc"},"range":[0,8033]}
data = requests.post(url, json = payload)

In [10]:
stocks_data = data.json()['data']
len(stocks_data)

8033

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [11]:
stocks_data[0]

{'s': 'NYSE:A',
 'd': ['agilent-technologies',
  'A',
  143.93,
  0.97516487,
  -0.01389564,
  -0.0625,
  -0.16649324,
  -0.26335851,
  -0.11105713,
  1.59525658,
  1.93342776,
  1.38057336,
  1.74607663,
  -6.11832235,
  -6.5207508,
  10.89452192,
  -5.28428534,
  7.62730876,
  1314695.03333333,
  'Agilent Technologies, Inc.',
  'stock',
  'common',
  'delayed_streaming_900',
  100,
  1,
  'false',
  0,
  'USD',
  'USD']}

In [22]:
stats_from_tradingview = []

for data in stocks_data:
    stock = {}
    stock['Symbol'] = data['d'][1]
    stock['Name'] = data['d'][19]
    stock['Price'] = data['d'][2]
    stock['CHG From Open'] = data['d'][3]
    stock['1 min CHG'] = data['d'][4]
    stock['5 min CHG'] = data['d'][5]
    stock['15 min CHG'] = data['d'][6]
    stock['1 Hr CHG'] = data['d'][7]
    stock['4 Hr CHG'] = data['d'][8]
    stock['1 Day CHG'] = data['d'][9]
    stock['1 Week CHG'] = data['d'][10]
    stock['1 Month CHG'] = data['d'][11]
    stock['1 Week Performance'] = data['d'][12]
    stock['1 Month Performance'] = data['d'][13]
    stock['3 Month Performance'] = data['d'][14]
    stock['6 Month Performance'] = data['d'][15]
    stock['YTD Performance'] = data['d'][16]
    stock['1 Year Performance'] = data['d'][17]
    stock['AvgVolume'] = data['d'][18]
    
    stats_from_tradingview.append(stock)

In [24]:
stats_from_tradingview[0]['5 min CHG']

-0.0625

In [25]:
# Serializing json
json_object = json.dumps(stats_from_tradingview)
 
# Writing to sample.json
with open("stocks_all_performance_short_&_long.json", "w") as outfile:
    outfile.write(json_object)

In [26]:
hqm_dataframe = pd.read_json('stocks_all_performance_short_&_long.json')
available_stocks = pd.read_json('curated_stocks_all.json')
available_stocks_list = available_stocks['symbol'].tolist()

In [27]:
hqm_dataframe = hqm_dataframe[hqm_dataframe['Symbol'].isin(available_stocks_list)]

In [41]:
mean_val = hqm_dataframe['AvgVolume'].mean()
hqm_dataframe['AvgVolume'].fillna(value=mean_val, inplace=True)
mean_val = hqm_dataframe['1 Week CHG'].mean()
hqm_dataframe['1 Week CHG'].fillna(value=mean_val, inplace=True)
hqm_dataframe.isna().any()

Symbol                 False
Name                   False
Price                  False
CHG From Open          False
1 min CHG              False
5 min CHG              False
15 min CHG             False
1 Hr CHG               False
4 Hr CHG               False
1 Day CHG              False
1 Week CHG             False
1 Month CHG            False
1 Week Performance     False
1 Month Performance    False
3 Month Performance    False
6 Month Performance    False
YTD Performance        False
1 Year Performance     False
AvgVolume              False
dtype: bool

In [43]:
hqm_dataframe['AvgVolume'] = hqm_dataframe['AvgVolume'].apply(lambda x : numerize.numerize(x, 2))

In [44]:
hqm_dataframe

,Symbol,Name,Price,CHG From Open,1 min CHG,5 min CHG,15 min CHG,1 Hr CHG,4 Hr CHG,1 Day CHG,1 Week CHG,1 Month CHG,1 Week Performance,1 Month Performance,3 Month Performance,6 Month Performance,YTD Performance,1 Year Performance,AvgVolume
0,A,"Agilent Technologies, Inc.",143.9300,0.975165,-0.013896,-0.062500,-0.166493,-0.263359,-0.111057,1.595257,1.933428,1.380573,1.746077,-6.118322,-6.520751,10.894522,-5.284285,7.627309,1.31M
1,AA,Alcoa Corporation,55.5200,2.190318,-0.116938,-0.197735,0.072098,-0.071994,0.361533,3.408456,19.526372,13.445035,18.303857,6.564299,15.522264,12.662338,21.302163,-32.727493,5.22M
10,AAMC,Altisource Asset Management Corp Com,53.2001,-1.953373,0.000188,0.000188,0.000188,0.000188,-1.316641,-0.949358,28.502657,2.288214,32.962356,35.714541,180.000526,171.192480,166.000500,172.821026,8.91K
11,AAME,Atlantic American Corporation,2.3899,-6.278431,-4.404000,-4.404000,-4.404000,-4.404000,-1.072109,-3.242915,-0.420833,-4.404000,-3.242915,-4.784861,-17.304498,-17.017361,13.804762,-6.278431,4.12K
12,AAN,"Aarons Holdings Company, Inc.",12.7800,3.398058,0.589854,0.788022,0.669028,0.392465,-0.544323,3.314470,-9.554140,-10.940767,-7.391304,-12.164948,6.500000,4.241436,5.098684,-41.909091,375.18K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8025,ZUO,"Zuora, Inc.",10.0200,2.244898,0.000000,0.100000,-0.149626,0.451581,0.958144,4.049844,16.241299,18.299882,15.040184,26.515152,27.157360,22.493888,55.348837,-36.015326,1.15M
8026,ZVIA,Zevia PBC,3.7500,0.806452,0.000000,0.806452,1.902174,2.040816,2.599179,1.351351,-0.265957,7.449857,-8.088235,-14.965986,-14.578588,-14.187643,-8.536585,-38.118812,143.84K
8030,ZYME,Zymeworks Inc.,8.0100,0.125000,0.125000,0.000000,-0.124688,0.062461,0.000000,0.125000,-7.076566,-2.554745,-6.315789,-15.684211,3.622251,22.290076,0.754717,14.921090,1.09M
8031,ZYNE,"Zynerba Pharmaceuticals, Inc.",0.4962,5.574468,0.282400,-0.051875,1.771532,1.771532,-2.071647,1.472393,-4.576923,7.402597,-0.760000,-22.480862,-11.392857,-56.852174,-8.887257,-75.676471,314.38K


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

## Cleaning Data
Clean all rows that have no historical Price Return, as in Nonetype

In [45]:
# for row in hqm_dataframe.index:
#     if(hqm_dataframe.loc[row, '1yearPerformace'] == None):
#         hqm_dataframe.drop(row, inplace=True)

hqm_dataframe.reset_index(inplace=True, drop=True)

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [46]:
hqm_dataframe

,Symbol,Name,Price,CHG From Open,1 min CHG,5 min CHG,15 min CHG,1 Hr CHG,4 Hr CHG,1 Day CHG,1 Week CHG,1 Month CHG,1 Week Performance,1 Month Performance,3 Month Performance,6 Month Performance,YTD Performance,1 Year Performance,AvgVolume
0,A,"Agilent Technologies, Inc.",143.9300,0.975165,-0.013896,-0.062500,-0.166493,-0.263359,-0.111057,1.595257,1.933428,1.380573,1.746077,-6.118322,-6.520751,10.894522,-5.284285,7.627309,1.31M
1,AA,Alcoa Corporation,55.5200,2.190318,-0.116938,-0.197735,0.072098,-0.071994,0.361533,3.408456,19.526372,13.445035,18.303857,6.564299,15.522264,12.662338,21.302163,-32.727493,5.22M
2,AAMC,Altisource Asset Management Corp Com,53.2001,-1.953373,0.000188,0.000188,0.000188,0.000188,-1.316641,-0.949358,28.502657,2.288214,32.962356,35.714541,180.000526,171.192480,166.000500,172.821026,8.91K
3,AAME,Atlantic American Corporation,2.3899,-6.278431,-4.404000,-4.404000,-4.404000,-4.404000,-1.072109,-3.242915,-0.420833,-4.404000,-3.242915,-4.784861,-17.304498,-17.017361,13.804762,-6.278431,4.12K
4,AAN,"Aarons Holdings Company, Inc.",12.7800,3.398058,0.589854,0.788022,0.669028,0.392465,-0.544323,3.314470,-9.554140,-10.940767,-7.391304,-12.164948,6.500000,4.241436,5.098684,-41.909091,375.18K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3160,ZUO,"Zuora, Inc.",10.0200,2.244898,0.000000,0.100000,-0.149626,0.451581,0.958144,4.049844,16.241299,18.299882,15.040184,26.515152,27.157360,22.493888,55.348837,-36.015326,1.15M
3161,ZVIA,Zevia PBC,3.7500,0.806452,0.000000,0.806452,1.902174,2.040816,2.599179,1.351351,-0.265957,7.449857,-8.088235,-14.965986,-14.578588,-14.187643,-8.536585,-38.118812,143.84K
3162,ZYME,Zymeworks Inc.,8.0100,0.125000,0.125000,0.000000,-0.124688,0.062461,0.000000,0.125000,-7.076566,-2.554745,-6.315789,-15.684211,3.622251,22.290076,0.754717,14.921090,1.09M
3163,ZYNE,"Zynerba Pharmaceuticals, Inc.",0.4962,5.574468,0.282400,-0.051875,1.771532,1.771532,-2.071647,1.472393,-4.576923,7.402597,-0.760000,-22.480862,-11.392857,-56.852174,-8.887257,-75.676471,314.38K


In [47]:
list(hqm_dataframe.columns.values)

['Symbol',
 'Name',
 'Price',
 'CHG From Open',
 '1 min CHG',
 '5 min CHG',
 '15 min CHG',
 '1 Hr CHG',
 '4 Hr CHG',
 '1 Day CHG',
 '1 Week CHG',
 '1 Month CHG',
 '1 Week Performance',
 '1 Month Performance',
 '3 Month Performance',
 '6 Month Performance',
 'YTD Performance',
 '1 Year Performance',
 'AvgVolume']

In [48]:
time_periods = [
    'CHG From Open',
    '1 min CHG',
    '5 min CHG',
    '15 min CHG',
    '1 Hr CHG',
    '4 Hr CHG',
    '1 Day CHG',
    '1 Week CHG',
    '1 Month CHG',
    '1 Week Performance',
    '1 Month Performance',
    '3 Month Performance',
    '6 Month Performance',
    'YTD Performance',
    '1 Year Performance'
]


for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period}'
        percentile_col = f'{time_period} Percentile'
        hqm_dataframe.loc[row, percentile_col] = stats.percentileofscore(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])

hqm_dataframe

,Symbol,Name,Price,CHG From Open,1 min CHG,5 min CHG,15 min CHG,1 Hr CHG,4 Hr CHG,1 Day CHG,...,4 Hr CHG Percentile,1 Day CHG Percentile,1 Week CHG Percentile,1 Month CHG Percentile,1 Week Performance Percentile,1 Month Performance Percentile,3 Month Performance Percentile,6 Month Performance Percentile,YTD Performance Percentile,1 Year Performance Percentile
0,A,"Agilent Technologies, Inc.",143.9300,0.975165,-0.013896,-0.062500,-0.166493,-0.263359,-0.111057,1.595257,...,35.545024,60.726698,54.249605,51.437599,51.974724,31.974724,30.521327,66.287520,19.336493,75.071090
1,AA,Alcoa Corporation,55.5200,2.190318,-0.116938,-0.197735,0.072098,-0.071994,0.361533,3.408456,...,61.421801,83.949447,97.630332,97.503949,97.251185,80.789889,76.082148,68.973144,76.872038,30.078989
2,AAMC,Altisource Asset Management Corp Com,53.2001,-1.953373,0.000188,0.000188,0.000188,0.000188,-1.316641,-0.949358,...,9.731438,10.458136,99.083728,63.601896,99.304897,98.515008,99.589258,99.778831,99.526066,99.462875
3,AAME,Atlantic American Corporation,2.3899,-6.278431,-4.404000,-4.404000,-4.404000,-4.404000,-1.072109,-3.242915,...,11.785150,3.317536,31.469194,9.036335,16.714060,37.251185,13.080569,26.161137,64.960506,57.946288
4,AAN,"Aarons Holdings Company, Inc.",12.7800,3.398058,0.589854,0.788022,0.669028,0.392465,-0.544323,3.314470,...,20.631912,83.443918,4.265403,2.369668,7.109005,18.420221,61.042654,56.840442,44.612954,24.360190
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3160,ZUO,"Zuora, Inc.",10.0200,2.244898,0.000000,0.100000,-0.149626,0.451581,0.958144,4.049844,...,83.823065,87.788310,96.587678,98.704581,95.829384,97.314376,86.445498,80.663507,94.913112,28.278041
3161,ZVIA,Zevia PBC,3.7500,0.806452,0.000000,0.806452,1.902174,2.040816,2.599179,1.351351,...,95.513428,55.734597,32.890995,91.469194,6.508689,13.807267,15.892575,29.415482,13.396524,26.571880
3162,ZYME,Zymeworks Inc.,8.0100,0.125000,0.125000,0.000000,-0.124688,0.062461,0.000000,0.125000,...,41.753555,25.655608,5.845182,13.554502,8.894155,12.764613,54.691943,80.315956,33.522907,81.864139
3163,ZYNE,"Zynerba Pharmaceuticals, Inc.",0.4962,5.574468,0.282400,-0.051875,1.771532,1.771532,-2.071647,1.472393,...,4.960506,58.420221,9.383886,91.279621,30.616114,6.729858,20.410742,6.350711,12.796209,7.045814


45.67404426559356

## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [50]:
list(hqm_dataframe.columns.values)

['Symbol',
 'Name',
 'Price',
 'CHG From Open',
 '1 min CHG',
 '5 min CHG',
 '15 min CHG',
 '1 Hr CHG',
 '4 Hr CHG',
 '1 Day CHG',
 '1 Week CHG',
 '1 Month CHG',
 '1 Week Performance',
 '1 Month Performance',
 '3 Month Performance',
 '6 Month Performance',
 'YTD Performance',
 '1 Year Performance',
 'AvgVolume',
 'CHG From Open Percentile',
 '1 min CHG Percentile',
 '5 min CHG Percentile',
 '15 min CHG Percentile',
 '1 Hr CHG Percentile',
 '4 Hr CHG Percentile',
 '1 Day CHG Percentile',
 '1 Week CHG Percentile',
 '1 Month CHG Percentile',
 '1 Week Performance Percentile',
 '1 Month Performance Percentile',
 '3 Month Performance Percentile',
 '6 Month Performance Percentile',
 'YTD Performance Percentile',
 '1 Year Performance Percentile']

In [52]:
short_term_period = [
    'CHG From Open',
    '1 min CHG',
    '5 min CHG',
    '15 min CHG',
    '1 Hr CHG',
    '4 Hr CHG',
    '1 Day CHG',
    '1 Week CHG',
    '1 Month CHG'
]
long_term_period = [
    '1 Week Performance',
    '1 Month Performance',
    '3 Month Performance',
    '6 Month Performance',
    'YTD Performance',
    '1 Year Performance'
]


In [53]:
# HQM Short Term
for row in hqm_dataframe.index:
    momentum_percentile = []
    for time_period in short_term_period:
        momentum_percentile.append(hqm_dataframe.loc[row, f'{time_period} Percentile'])
    hqm_dataframe.loc[row, 'HQM Short'] = mean(momentum_percentile)

# HQM Long Term
for row in hqm_dataframe.index:
    momentum_percentile = []
    for time_period in long_term_period:
        momentum_percentile.append(hqm_dataframe.loc[row, f'{time_period} Percentile'])
    hqm_dataframe.loc[row, 'HQM Long'] = mean(momentum_percentile)

    
# HQM Score
for row in hqm_dataframe.index:
    momentum_percentile = []
    for time_period in time_periods:
        momentum_percentile.append(hqm_dataframe.loc[row, f'{time_period} Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentile)  

In [61]:
hqm_dataframe
hqm_dataframe['HQM Diff'] = hqm_dataframe['HQM Long'] - hqm_dataframe['HQM Short']
hqm_dataframe['HQM Diff'] = hqm_dataframe['HQM Diff'].apply(abs)

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [56]:
hqm_dataframe.sort_values('HQM Score', ascending=False, inplace=True)
hqm_dataframe.reset_index(inplace=True, drop=True)
# hqm_dataframe[hqm_dataframe['Price'] < 10][:50]

In [57]:
cols = list(hqm_dataframe.columns.values)
cols

['Symbol',
 'Name',
 'Price',
 'CHG From Open',
 '1 min CHG',
 '5 min CHG',
 '15 min CHG',
 '1 Hr CHG',
 '4 Hr CHG',
 '1 Day CHG',
 '1 Week CHG',
 '1 Month CHG',
 '1 Week Performance',
 '1 Month Performance',
 '3 Month Performance',
 '6 Month Performance',
 'YTD Performance',
 '1 Year Performance',
 'AvgVolume',
 'CHG From Open Percentile',
 '1 min CHG Percentile',
 '5 min CHG Percentile',
 '15 min CHG Percentile',
 '1 Hr CHG Percentile',
 '4 Hr CHG Percentile',
 '1 Day CHG Percentile',
 '1 Week CHG Percentile',
 '1 Month CHG Percentile',
 '1 Week Performance Percentile',
 '1 Month Performance Percentile',
 '3 Month Performance Percentile',
 '6 Month Performance Percentile',
 'YTD Performance Percentile',
 '1 Year Performance Percentile',
 'HQM Short',
 'HQM Long',
 'HQM Score',
 'HQM Diff']

In [58]:
# rearrange columns
cols = list(hqm_dataframe.columns.values)
hqm_dataframe = hqm_dataframe[[
    'Symbol',
    'Name',
    'Price',
    'HQM Score',
    'HQM Short',
    'HQM Long',
    'HQM Diff',
    'AvgVolume',
    'CHG From Open',
    '1 min CHG',
    '5 min CHG',
    '15 min CHG',
    '1 Hr CHG',
    '4 Hr CHG',
    '1 Day CHG',
    '1 Week CHG',
    '1 Month CHG',
    '1 Week Performance',
    '1 Month Performance',
    '3 Month Performance',
    '6 Month Performance',
    'YTD Performance',
    '1 Year Performance',
    'CHG From Open Percentile',
    '1 min CHG Percentile',
    '5 min CHG Percentile',
    '15 min CHG Percentile',
    '1 Hr CHG Percentile',
    '4 Hr CHG Percentile',
    '1 Day CHG Percentile',
    '1 Week CHG Percentile',
    '1 Month CHG Percentile',
    '1 Week Performance Percentile',
    '1 Month Performance Percentile',
    '3 Month Performance Percentile',
    '6 Month Performance Percentile',
    'YTD Performance Percentile',
    '1 Year Performance Percentile'
]]

In [68]:
hqm_dataframe[:50]

,Symbol,Name,Price,HQM Score,HQM Short,HQM Long,HQM Diff,AvgVolume,CHG From Open,1 min CHG,...,4 Hr CHG Percentile,1 Day CHG Percentile,1 Week CHG Percentile,1 Month CHG Percentile,1 Week Performance Percentile,1 Month Performance Percentile,3 Month Performance Percentile,6 Month Performance Percentile,YTD Performance Percentile,1 Year Performance Percentile
0,VTNR,"Vertex Energy, Inc",11.0800,94.535018,93.594874,95.945234,2.350360,2.76M,5.927342,0.180832,...,94.565561,94.407583,99.557662,98.451817,99.431280,99.304897,93.333333,89.826224,97.756714,96.018957
1,PLX,"Protalix BioTherapeutics, Inc. (DE)",2.0500,94.304371,92.181850,97.488152,5.306302,563.5K,3.535354,0.487805,...,96.808847,84.660348,94.249605,86.066351,97.630332,98.167457,98.420221,97.788310,93.775671,99.146919
2,YEXT,"Yext, Inc.",8.7400,93.985255,94.461998,93.270142,1.191855,1M,7.239264,0.228833,...,92.859400,96.840442,99.052133,98.830964,98.988942,97.187994,92.511848,98.704581,87.045814,85.181675
3,WAVD,"WaveDancer, Inc.",1.2200,90.596103,99.015271,77.967351,21.047920,66.08K,46.987952,0.826446,...,99.936809,99.968404,99.873618,99.936809,99.873618,99.778831,98.925750,60.916272,99.589258,8.720379
4,ARAV,"Aravive, Inc.",2.1500,90.589784,94.760400,84.333860,10.426540,528.38K,14.361702,0.467290,...,99.715640,98.988942,95.955766,97.093207,57.219589,97.598736,96.587678,99.936809,96.587678,58.072670
5,ZEUS,"Olympic Steel, Inc.",58.7500,90.281201,84.914867,98.330700,13.415833,96.37K,3.524229,0.051081,...,95.829384,86.413902,98.799368,96.524487,99.526066,98.199052,97.345972,99.178515,97.187994,98.546603
6,MATH,Metalpha Technology Holding Limited,0.9990,89.143760,90.468668,87.156398,3.312270,49.1K,5.457616,5.825762,...,99.431280,96.303318,92.764613,32.069510,94.786730,99.810427,98.578199,87.203791,97.914692,44.644550
7,IE,Ivanhoe Electric Inc.,15.8300,88.648763,86.406881,92.011585,5.604704,240.62K,2.129032,0.157978,...,87.740916,86.445498,93.522907,84.644550,88.720379,96.145340,94.375987,95.545024,86.255924,91.026856
8,BWEN,"Broadwind, Inc.",4.8300,88.358083,94.488327,79.162717,15.325610,1.08M,13.647059,1.239669,...,93.996840,98.609795,88.751975,89.921011,60.410742,20.947867,99.557662,95.450237,99.620853,98.988942
9,PBPB,Potbelly Corporation,7.9400,87.905213,90.258031,84.375987,5.882043,67.7K,9.366391,0.128791,...,94.818325,97.567141,88.056872,95.260664,82.938389,54.944708,94.660348,91.279621,91.690363,90.742496


In [67]:
hqm_dataframe.sort_values('HQM Short', ascending=False)[:50]

,Symbol,Name,Price,HQM Score,HQM Short,HQM Long,HQM Diff,AvgVolume,CHG From Open,1 min CHG,...,4 Hr CHG Percentile,1 Day CHG Percentile,1 Week CHG Percentile,1 Month CHG Percentile,1 Week Performance Percentile,1 Month Performance Percentile,3 Month Performance Percentile,6 Month Performance Percentile,YTD Performance Percentile,1 Year Performance Percentile
3,WAVD,"WaveDancer, Inc.",1.220000,90.596103,99.015271,77.967351,21.047920,66.08K,46.987952,0.826446,...,99.936809,99.968404,99.873618,99.936809,99.873618,99.778831,98.925750,60.916272,99.589258,8.720379
11,MGLD,"The Marygold Companies, Inc.",1.750000,86.754081,98.053361,69.805161,28.248201,8.2K,8.803780,2.941176,...,99.494471,94.786730,94.533965,98.357030,96.240126,46.698262,95.197472,78.862559,71.090047,30.742496
13,AVAH,Aveanna Healthcare Holdings Inc.,1.450000,86.636124,97.930490,69.694576,28.235914,462.42K,14.173228,0.694444,...,96.398104,99.494471,94.849921,98.767773,94.881517,93.112164,98.546603,24.423381,98.546603,8.657188
4,ARAV,"Aravive, Inc.",2.150000,90.589784,94.760400,84.333860,10.426540,528.38K,14.361702,0.467290,...,99.715640,98.988942,95.955766,97.093207,57.219589,97.598736,96.587678,99.936809,96.587678,58.072670
8,BWEN,"Broadwind, Inc.",4.830000,88.358083,94.488327,79.162717,15.325610,1.08M,13.647059,1.239669,...,93.996840,98.609795,88.751975,89.921011,60.410742,20.947867,99.557662,95.450237,99.620853,98.988942
2,YEXT,"Yext, Inc.",8.740000,93.985255,94.461998,93.270142,1.191855,1M,7.239264,0.228833,...,92.859400,96.840442,99.052133,98.830964,98.988942,97.187994,92.511848,98.704581,87.045814,85.181675
375,EGIO,"Edgio, Inc.",1.300000,68.496051,93.745831,30.621380,63.124451,1.11M,6.557377,1.562500,...,93.443918,95.513428,79.968404,86.161137,69.352291,10.047393,18.704581,4.486572,70.774092,10.363349
0,VTNR,"Vertex Energy, Inc",11.080000,94.535018,93.594874,95.945234,2.350360,2.76M,5.927342,0.180832,...,94.565561,94.407583,99.557662,98.451817,99.431280,99.304897,93.333333,89.826224,97.756714,96.018957
153,EVGO,EVgo Inc.,6.370000,75.856767,93.294717,49.699842,43.594874,2.44M,6.521739,0.393391,...,90.584518,95.418641,89.794629,91.342812,77.883096,29.510269,60.410742,15.987362,88.720379,25.687204
241,MYMD,"MyMD Pharmaceuticals, Inc.",2.010000,71.820958,93.252589,39.673512,53.579077,237.81K,5.235602,0.497512,...,98.135861,96.145340,80.315956,84.360190,63.191153,5.023697,40.726698,10.584518,97.030016,21.484992
